<a href="https://colab.research.google.com/github/alfrizzle/NLP-Projects/blob/master/SM_Training_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import os

In [28]:
os.chdir('/content/drive/MyDrive/datasets')

In [29]:
df = pd.read_csv('wiki_attacks_sample.csv')

In [30]:
df.shape

(1000, 2)

In [34]:
df.to_json('wiki_attacks_sample.json')

In [7]:
# df_sample = df[:1000]

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
train_df, test_df = train_test_split(df, test_size = .2, stratify=df['label'])

In [33]:
print(train_df.shape)
print(test_df.shape)

(800, 2)
(200, 2)


In [39]:
test_df.label.value_counts()

0    188
1     12
Name: label, dtype: int64

In [30]:
train_df.to_json('wiki_attacks_train.json')

In [31]:
test_df.to_json('wiki_attacks_test.json')

In [20]:
json_df = pd.read_json('wiki_attacks_sample_train.json')

In [22]:
json_df.head()

,input,label
261,"== VfD == No, I voted just after seeing you...",0
113,==To the person who will rewrite the essay...,0
102,Wow! That was fast. I'm impressed.,0
959,putting an accuracy warning on it,0
874,I think that is actually useful information th...,0


In [ ]:
df_sample.to_csv('wiki_attacks_sample.csv', index=False)

## Functions

In [36]:
file = 'wiki_attacks_sample.csv'

In [39]:
df = read_object(file, 'csv')

In [40]:
df.shape

(1000, 2)

# Start Here

In [1]:
!pip install torch
!pip install transformers

     |████████████████████████████████| 3.4 MB 5.5 MB/s 
     |████████████████████████████████| 895 kB 44.0 MB/s 
     |████████████████████████████████| 3.3 MB 37.7 MB/s 
     |████████████████████████████████| 61 kB 287 kB/s 
     |████████████████████████████████| 596 kB 41.5 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import argparse
import os

# import boto3
import numpy as np
import pandas as pd
from sklearn.metrics import precision_score, recall_score, average_precision_score, roc_auc_score, f1_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler
from transformers import AdamW, AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

In [3]:
def read_object(file_path, file_type):
    """Read a file from indicated file path into a pandas.DataFrame.
    Arguments:
        file_path (str)
    Returns:
        df (pd.DataFrame)
    """
    # Get S3 client.
    # s3 = boto3.client('s3')
    # Get object.
    # bucket, key = s3_path.split('/')[2], '/'.join(s3_path.split('/')[3:])
    # object = s3.get_object(Bucket=bucket, Key=key)
    # Read object into pandas.DataFrame.
    if file_type == "csv":
        df = pd.read_csv(file_path)
    elif file_type == "json":
        df = pd.read_json(file_path, lines=True)
    else:
        raise Exception("Only file_type csv and json currently supported.")
    return df

In [4]:
class ClassifierDataset(Dataset):
    """Inherits from torch.utils.data.Dataset.
    Arguments:
        data (pd.DataFrame): Contains columns 'input' and 'label'.
        model_name (str): Name of the transformers model to load the tokenizer.
        max_sequence_length (int): Maximum length of the sequence.
    """
    def __init__(self, data, model_name, max_sequence_length):
        self.data = data
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.max_sequence_length = max_sequence_length

    def __getitem__(self, index):
        """Returns the model inputs at the specified index."""
        text_input = self.data["input"].iloc[index]
        input_dict = self.tokenizer.encode_plus(
            text_input,
            max_length=max_sequence_length,
            padding="max_length",
            truncation=True,
            return_tensors="pt"
        )
        inputs = {
            "input_ids": input_dict["input_ids"][0],
            "attention_mask": input_dict["attention_mask"][0],
            "labels": torch.tensor(self.data["label"].iloc[index], dtype=torch.long)
        }
        return inputs

    def __len__(self):
        """Returns the length of the dataset."""
        return len(self.data)

In [4]:
# class ClassifierDataset(Dataset):
#     """Inherits from torch.utils.data.Dataset.
#     Arguments:
#         data (pd.DataFrame): Contains columns 'input' and 'label'.
#         model_name (str): Name of the transformers model to load the tokenizer.
#         max_sequence_length (int): Maximum length of the sequence.
#     """
#     def __init__(self, data, model_name, max_sequence_length):
#         self.data = data
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.max_sequence_length = max_sequence_length

#     def __getitem__(self, index):
#         """Returns the model inputs at the specified index."""
#         text_input = self.data["input"].iloc[index]
#         input_dict = self.tokenizer.encode_plus(
#             text_input,
#             max_length=args.max_sequence_length,
#             padding="max_length",
#             truncation=True,
#             return_tensors="pt"
#         )
#         inputs = {
#             "input_ids": input_dict["input_ids"][0],
#             "attention_mask": input_dict["attention_mask"][0],
#             "labels": torch.tensor(self.data["label"].iloc[index], dtype=torch.long)
#         }
#         return inputs

#     def __len__(self):
#         """Returns the length of the dataset."""
#         return len(self.data)


# def preprocess_data(df):
#     """Pre-process data for training."""
#     # Convert 5 star scale to binary -- positive (1) or negative (0).
#     star_map = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
#     df["label"] = df["star_rating"].map(star_map)
#     # Concatenate the review headline with the body for model input.
#     df["input"] = df["review_headline"] + " " + df["review_body"]
#     # Drop nans.
#     df = df.loc[df["input"].notnull()].copy()
#     # Split data into train and valid.
#     traindf, validdf = train_test_split(df[["input", "label"]])
#     return traindf, validdf


# def compute_metrics(output):
#     """Compute custom metrics to track in logging."""
#     y_true = output.label_ids
#     y_pred = output.predictions.argmax(-1)
#     precision = precision_score(y_true, y_pred)
#     recall = recall_score(y_true, y_pred)
#     auprc = average_precision_score(y_true, y_pred)
#     return {
#         "precision": precision,
#         "recall": recall,
#         "auprc": auprc
#     }


# def main(args):
#     """Executes training job."""
#     # Load data into a pandas DataFrame.
#     df = read_object(args.input_path, file_type="json").sample(frac=1.)
#     if args.max_data_rows:
#         df = df.iloc[:args.max_data_rows].copy()
#     print(f"Data contains {len(df)} rows")
#     # Preprocess and featurize data.
#     traindf, validdf = preprocess_data(df)
#     # Create data set for model input.
#     train_dataset = ClassifierDataset(traindf, args.model_name, args.max_sequence_length)
#     valid_dataset = ClassifierDataset(validdf, args.model_name, args.max_sequence_length)
#     print(train_dataset[0])
#     # Define model and trainer.
#     model = AutoModelForSequenceClassification.from_pretrained(
#         args.model_name,
#         num_labels=2,
#         torchscript=True
#     )
#     training_args = TrainingArguments(
#         output_dir=os.path.join(args.model_dir, "results"),
#         num_train_epochs=args.epochs,
#         per_device_train_batch_size=args.train_batch_size,
#         per_device_eval_batch_size=args.valid_batch_size,
#         learning_rate=args.learning_rate,
#         adam_epsilon=args.adam_epsilon,
#         warmup_steps=500,
#         weight_decay=args.weight_decay,
#         logging_dir=os.path.join(args.model_dir, "logs"),
#         logging_steps=10,
#         evaluation_strategy="steps",
#         load_best_model_at_end=True
#     )
#     trainer = Trainer(
#         model=model,
#         args=training_args,
#         train_dataset=train_dataset,
#         eval_dataset=valid_dataset,
#         compute_metrics=compute_metrics
#     )
#     # Train the model.
#     trainer.train()
#     trainer.evaluate()
#     # Save the model to the output_path defined in train_model.py.
#     device = torch.device("cuda")
#     dummy_row = train_dataset[0]
#     dummy_input = (
#         dummy_row["input_ids"].unsqueeze(0).to(device),
#         dummy_row["attention_mask"].unsqueeze(0).to(device)
#     )
#     traced_model = torch.jit.trace(model, dummy_input)
#     torch.jit.save(traced_model, os.path.join(args.model_dir, "model.pth"))


# if __name__ =='__main__':
#     parser = argparse.ArgumentParser()
#     # Hyperparameters from launch_training_job.py get passed in as command line args.
#     parser.add_argument('--input_path', type=str)
#     parser.add_argument('--train_size', type=float, default=.85)
#     parser.add_argument('--adam_epsilon', type=float, default=1e-8)
#     parser.add_argument('--epochs', type=int, default=2)
#     parser.add_argument('--learning_rate', type=float, default=5e-5)
#     parser.add_argument('--weight_decay', type=float, default=0.0)
#     parser.add_argument('--max_data_rows', type=int, default=None)
#     parser.add_argument('--max_sequence_length', type=int, default=128)
#     parser.add_argument('--model_name', type=str, default='distilbert-base-uncased')
#     parser.add_argument('--train_batch_size', type=int, default=16)
#     parser.add_argument('--valid_batch_size', type=int, default=128)
#     # SageMaker environment variables.
#     parser.add_argument('--hosts', type=list, default=os.environ['SM_HOSTS'])
#     parser.add_argument('--current_host', type=str, default=os.environ['SM_CURRENT_HOST'])
#     parser.add_argument('--model_dir', type=str, default=os.environ['SM_MODEL_DIR']) # output_path arg from train_model.py.
#     parser.add_argument('--num_cpus', type=int, default=os.environ['SM_NUM_CPUS'])
#     parser.add_argument('--num_gpus', type=int, default=os.environ['SM_NUM_GPUS'])
#     # Parse command-line args and run main.
#     args = parser.parse_args()
#     main(args)

In [5]:
os.chdir('/content/drive/MyDrive/sm_training/models/distilroberta-base')
!pwd

/content/drive/MyDrive/sm_training/models/distilroberta-base


In [6]:
# Set args
# first change working directory to '/content/drive/MyDrive/sm_training'

# model_name = 'distilbert-base-uncased'
model_name = 'distilroberta-base'
max_sequence_length = 128
train_df = '/content/drive/MyDrive/datasets/wiki_attacks.csv'
test_df = '/content/drive/MyDrive/datasets/wiki_attacks.csv'
train_file = '/content/drive/MyDrive/datasets/wiki_attacks.csv'
output_dir = 'results'
epochs = 2
train_batch_size = 32
valid_batch_size = 128
learning_rate = 5e-5
adam_epsilon = 1e-8
weight_decay = 0.0
logging_dir = 'logs'


In [7]:
def preprocess_data(df, file_type):
    """Pre-process data for training."""
    # # Convert 5 star scale to binary -- positive (1) or negative (0).
    # star_map = {1: 0, 2: 0, 3: 0, 4: 1, 5: 1}
    # df["label"] = df["star_rating"].map(star_map)
    # # Concatenate the review headline with the body for model input.
    # df["input"] = df["review_headline"] + " " + df["review_body"]
    # # Drop nans.
    # df = df.loc[df["input"].notnull()].copy()
    # # Split data into train and valid.
    # traindf, validdf = train_test_split(df[["input", "label"]])
    df = read_object(df, file_type)
    traindf, testdf = train_test_split(df, test_size=0.2, stratify=df['label'])
    return traindf, testdf

In [8]:
traindf, testdf = preprocess_data(train_file, 'csv')

In [9]:
traindf.head()

,input,label
10268,== Deletion review sockpuppet question == A...,0
6126,`:Thank you for your interest in my opinion. I...,0
4551,:I collect old Pears Cyclopaedias. I'm just ...,0
18071,` == Warnings == Don't you think a uw-create...,0
3652,"When someone points out a link, you could try ...",0


In [10]:
traindf.label.value_counts()

0    14251
1     1749
Name: label, dtype: int64

In [11]:
train_dataset = ClassifierDataset(traindf, model_name, max_sequence_length)
test_dataset = ClassifierDataset(testdf, model_name, max_sequence_length)

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [12]:
train_dataset[0]

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0]),
 'input_ids': tensor([    0,  1437, 45994,   926,  2716,  1499,  1551, 37390,   642, 43317,
           864, 45994,  1437,    83, 43762,  1551,   630,    75,  1341,  2045,
           101,     5,   317,     7,  2268,   549,    50,    45,    38,   348,
            56,   277,  2349,    53,     6, 11380,     6,    38,   348,    56,
            55,    87,    65,  1316,     6,   600,    38,   348,   393,   341,
           106,     7,   323,    50,   190, 10754,    19,   349,    97,     8,
            98,    38,   218,    75,   679,    38, 1418

In [44]:
# Define model and trainer.
model = AutoModelForSequenceClassification.from_pretrained(
    model_name,
    num_labels=2,
    torchscript=True
    )

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torchscript": true,
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file htt

In [48]:
training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=epochs,
        per_device_train_batch_size=train_batch_size,
        per_device_eval_batch_size=valid_batch_size,
        learning_rate=learning_rate,
        adam_epsilon=adam_epsilon,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir=logging_dir,
        logging_steps=250,
        evaluation_strategy="steps",
        load_best_model_at_end=True
    )

using `logging_steps` to initialize `eval_steps` to 250
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [49]:
def compute_metrics(output):
    """Compute custom metrics to track in logging."""
    y_true = output.label_ids
    y_pred = output.predictions.argmax(-1)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    # auprc = average_precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    roc_auc = roc_auc_score(y_true, y_pred, average=None)
    return {
        "precision": precision,
        "recall": recall,
        "f1_score": f1,
        "roc_auc": roc_auc
    }

In [50]:
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset,
        compute_metrics=compute_metrics
    )

In [51]:
# # Train the model.
# trainer.train()
# trainer.evaluate()

# # Save the model to the output_path defined in train_model.py.
# device = torch.device("cuda")
# dummy_row = train_dataset[0]
# dummy_input = (
#     dummy_row["input_ids"].unsqueeze(0).to(device),
#     dummy_row["attention_mask"].unsqueeze(0).to(device)
#     )
# traced_model = torch.jit.trace(model, dummy_input)
# torch.jit.save(traced_model, os.path.join(args.model_dir, "model.pth"))

## Start Training

In [52]:
trainer.train()

***** Running training *****
  Num examples = 16000
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 1000


Step,Training Loss,Validation Loss,Precision,Recall,F1 Score,Roc Auc
250,0.287500,0.154683,0.835484,0.592677,0.693440,0.789182
500,0.159500,0.184604,0.668529,0.821510,0.737166,0.885776
750,0.138500,0.154733,0.701581,0.812357,0.752916,0.884988
1000,0.116300,0.141084,0.791667,0.739130,0.764497,0.857637


***** Running Evaluation *****
  Num examples = 4000
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 128
Saving model checkpoint to results/checkpoint-500
Configuration saved in results/checkpoint-500/config.json
Model weights saved in results/checkpoint-500/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 128
***** Running Evaluation *****
  Num examples = 4000
  Batch size = 128
Saving model checkpoint to results/checkpoint-1000
Configuration saved in results/checkpoint-1000/config.json
Model weights saved in results/checkpoint-1000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1000 (score: 0.14108391106128693).


TrainOutput(global_step=1000, training_loss=0.17546056747436523, metrics={'train_runtime': 810.1418, 'train_samples_per_second': 39.499, 'train_steps_per_second': 1.234, 'total_flos': 1059739189248000.0, 'train_loss': 0.17546056747436523, 'epoch': 2.0})

In [53]:
# trainer.evaluate()
eval_result = trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 4000
  Batch size = 128


In [54]:
eval_result

{'epoch': 2.0,
 'eval_f1_score': 0.7644970414201182,
 'eval_loss': 0.14108391106128693,
 'eval_precision': 0.7916666666666666,
 'eval_recall': 0.7391304347826086,
 'eval_roc_auc': 0.8576370669562777,
 'eval_runtime': 43.9869,
 'eval_samples_per_second': 90.936,
 'eval_steps_per_second': 0.727}

### Eval Test Results Testing

In [102]:
for key, value in eval_result.items():
    print(key, ' : ', value)

eval_loss  :  0.10770474374294281
eval_precision  :  0.6153846153846154
eval_recall  :  0.6666666666666666
eval_f1_score  :  0.64
eval_roc_auc  :  0.8200354609929078
eval_runtime  :  1.778
eval_samples_per_second  :  112.486
eval_steps_per_second  :  1.125
epoch  :  4.0


In [115]:
dict1 = eval_result
dict2 = dict()

for key in dict1:
    if key == 'eval_precision':
        dict2[key] = 'Precision'
    else:
        dict2[key] = dict1[key]

print(dict1)

# dict1 = dict2 #if you want to have it in original dict

{'eval_loss': 0.10770474374294281, 'eval_precision': 0.6153846153846154, 'eval_recall': 0.6666666666666666, 'eval_f1_score': 0.64, 'eval_roc_auc': 0.8200354609929078, 'eval_runtime': 1.778, 'eval_samples_per_second': 112.486, 'eval_steps_per_second': 1.125, 'epoch': 4.0}


In [110]:
def rename(old_dict,old_name,new_name):
    new_dict = {}
    for key,value in zip(old_dict.keys(),old_dict.values()):
        new_key = key if key != old_name else new_name
        new_dict[new_key] = old_dict[key]
    return new_dict

In [112]:
rename(partial_dict, 'eval_recall', 'Recall')

{'Recall': 0.6666666666666666,
 'eval_f1_score': 0.64,
 'eval_precision': 0.6153846153846154,
 'eval_roc_auc': 0.8200354609929078}

In [109]:
names_key = { 'eval_loss'           : 'Loss' ,
              'eval_precision'      : 'Precision' ,
              'eval_recall'         : 'Recall', 
              'eval_f1_score'       : 'f1 score',
              'eval_roc_auc'        : 'ROC_AUC',
              'eval_runtime'            : 'Runtime',
              'eval_roc_auc'            : 'ROC_AUC',
              'eval_samples_per_second' : 'samples_per_second',
              'eval_steps_per_second' : 'steps_per_second',
              'epochs'                : 'EPOCHS'
              }

for row in eval_result:
  for k, v in names_key.items():
    for old_name in row:
      if k == old_name:
        row[v] = row.pop(old_name)

print(eval_result)

{'eval_loss': 0.10770474374294281, 'eval_precision': 0.6153846153846154, 'eval_recall': 0.6666666666666666, 'eval_f1_score': 0.64, 'eval_roc_auc': 0.8200354609929078, 'eval_runtime': 1.778, 'eval_samples_per_second': 112.486, 'eval_steps_per_second': 1.125, 'epoch': 4.0}


In [106]:
keep = ['eval_precision', 'eval_recall', 'eval_f1_score', 'eval_roc_auc']

partial_dict = {k: v for k, v in eval_result.items() if k in keep}

new_keys = ['Precision','Recall','f1', 'ROC_AUC']
old_keys = ['eval_precision', 'eval_recall', 'eval_f1_score', 'eval_roc_auc']

partial_dict[new_keys] = partial_dict.pop(old_keys)

# for key, value in partial_dict.items():
#     print(key, ' : ', value)

# partial_dict = {"Precision" if k == 'eval_precision' else 
#                 k: v for k, v in eval_result.items() if k in keep}

# partial_dict = {"Recall" if k == 'eval_recall' else 
#                 k: v for k, v in eval_result.items() if k in keep}

# partial_dict = {"f1" if k == 'eval_f1_score' else 
#                 k: v for k, v in partial_dict.items() if k in keep}

# partial_dict = {"ROC_AUC" if k == 'eval_roc_auc' else 
#                 k: v for k, v in partial_dict.items() if k in keep}
# {"Precision" if k == 'eval_precision' else k:v for k,v in d.items()}

print(partial_dict)

TypeError: ignored

In [56]:
import json

# # Serializing json  
# json_object = json.dumps(eval_result) 
# print(json_object)

keep = ['eval_loss', 'eval_precision', 'eval_recall', 'eval_f1_score', 'eval_roc_auc']
partial_dict = {k: v for k, v in eval_result.items() if k in keep}

with open(os.path.join(output_data_dir, 'eval_results.json'), 'w') as fp:
          print('****** Eval Results ******')
          # for key, value in sorted(eval_result.items()):
          #   json.dump(eval_result, fp)
          json.dump(partial_dict, fp)

# keep = ['eval_precision', 'eval_recall', 'eval_f1_score', 'eval_roc_auc']

# partial_dict = {k: v for k, v in eval_result.items() if k in keep}
# print(partial_dict)

****** Eval Results ******


In [73]:
!pwd

/content/drive/My Drive/sm_training


In [55]:
output_data_dir = 'output_data'

In [75]:
# writes eval result to file which can be accessed later in s3 ouput
with open(os.path.join(output_data_dir, "eval_results.txt"), "w") as writer:
  print("***** Eval results *****")
  for key, value in sorted(eval_result.items()):
    writer.write(f"{key} = {value}\n")

***** Eval results *****


In [57]:
# Saves the model to s3
trainer.save_model(output_dir)

Saving model checkpoint to results
Configuration saved in results/config.json
Model weights saved in results/pytorch_model.bin


In [67]:
# Save the model to the output_path defined in train_model.py.
device = torch.device("cuda")
dummy_row = train_dataset[0]
dummy_input = (
    dummy_row["input_ids"].unsqueeze(0).to(device),
    dummy_row["attention_mask"].unsqueeze(0).to(device)
    )
traced_model = torch.jit.trace(model, dummy_input)
torch.jit.save(traced_model, os.path.join(output_dir, "model.pth"))